In this notebook, we make API requests to OpenRouter, to generate Gherkins from user stories using various models and various prompting approaches.

In [1]:
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import chardet # To detect file encodings
import asyncio
import httpx

import os

from datetime import datetime

load_dotenv()

or_token = os.getenv("openrouter_token")

In [2]:
# Check for user stories that are common between features , i.e. duplicates between files
folder_path = "./data"

# txt_files = [f for f in os.listdir(folder_path) if f.endswith("federalspending.txt")]
txt_files = [f for f in os.listdir(folder_path) if f.startswith("g04")]

user_stories_dict = {}

for file in txt_files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, "rb") as f:
        raw_data = f.read()
        detected_encoding = chardet.detect(raw_data)["encoding"]

        try:
            with open(file_path, "r", encoding=detected_encoding, errors="replace") as f:
                for line in f:
                    line = line.strip()
                    if line:
                        if line not in user_stories_dict:
                            user_stories_dict[line] = file
                        else:
                            if isinstance(user_stories_dict[line], list):
                                if file not in user_stories_dict[line]:
                                    print("Found actual duplicate between files")
                                    user_stories_dict[line].append(file)
                            else:
                                if user_stories_dict[line] != file:
                                    print("Found actual duplicate between files")
                                    user_stories_dict[line] = [user_stories_dict[line], file]

        except Exception as e:
            print(f"Error reading {file}: {e}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: './data'

In [3]:
user_stories_dict

{'As a user, I want to click on the address, so that it takes me to a new tab with Google Maps.': 'g04-recycling.txt',
 'As a user, I want to be able to anonymously view public information, so that I know about recycling centers near me before creating an account.': 'g04-recycling.txt',
 'As a user, I want to be able to enter my zip code and get a list of nearby recycling facilities, so that I can determine which ones I should consider.': 'g04-recycling.txt',
 'As a user, I want to be able to get the hours of each recycling facility, so that I can arrange drop-offs on my off days or during after-work hours.': 'g04-recycling.txt',
 'As a user, I want to have a flexible pick up time, so that I can more conveniently use the website.': 'g04-recycling.txt',
 'As a user, I want to be able to select different types of recyclable waste, so I have and get a list of facilities that accept each type and their opening hours, so that I can find an optimal route and schedule.': 'g04-recycling.txt',


In [4]:
# Check for list values in dict, rrepresenting user stories that appear in more than one file
shared_stories = any(isinstance(v, list) for v in user_stories_dict.values())

print(f"Shared user stories found: {shared_stories}")

Shared user stories found: False


In [5]:
# # Load user stories from text files - commented out as now loading into a dict to track duplicates 10/10/25
# folder_path = "./data"

# # txt_files = [f for f in os.listdir(folder_path) if f.endswith("federalspending.txt")]
# txt_files = [f for f in os.listdir(folder_path) if f.startswith("g04")]

# user_stories = []

# for file in txt_files:
#     file_path = os.path.join(folder_path, file)
#     with open(file_path, "rb") as f:
#         raw_data = f.read()
#         detected_encoding = chardet.detect(raw_data)["encoding"]

#         try:
#             with open(file_path, "r", encoding=detected_encoding, errors="replace") as f:
#                 for line in f:
#                     line = line.strip()
#                     if line:
#                         user_stories.append(line)

#         except Exception as e:
#             print(f"Error reading {file}: {e}")


In [6]:
# Load user stories from text files
folder_path = "./data"

# txt_files = [f for f in os.listdir(folder_path) if f.endswith("federalspending.txt")]
txt_files = [f for f in os.listdir(folder_path) if f.startswith("g04")]

user_stories = {}

for file in txt_files:
    file_path = os.path.join(folder_path, file)

    with open(file_path, "rb") as f:
        raw_data = f.read()
        detected_encoding = chardet.detect(raw_data)["encoding"]

        try:
            with open(file_path, "r", encoding=detected_encoding, errors="replace") as f:
                us_count = 1
                                
                for line in f:
                    line = line.strip()
                    if line:
                        # user_stories.append(line)
                        us_id = f"{file.split('-')[0]}_{us_count}"
                        user_stories[us_id] = line
                        us_count += 1


        except Exception as e:
            print(f"Error reading {file}: {e}")


In [7]:
len(user_stories)

51

In [8]:
user_stories

{'g04_1': 'As a user, I want to click on the address, so that it takes me to a new tab with Google Maps.',
 'g04_2': 'As a user, I want to be able to anonymously view public information, so that I know about recycling centers near me before creating an account.',
 'g04_3': 'As a user, I want to be able to enter my zip code and get a list of nearby recycling facilities, so that I can determine which ones I should consider.',
 'g04_4': 'As a user, I want to be able to get the hours of each recycling facility, so that I can arrange drop-offs on my off days or during after-work hours.',
 'g04_5': 'As a user, I want to have a flexible pick up time, so that I can more conveniently use the website.',
 'g04_6': 'As a user, I want to be able to select different types of recyclable waste, so I have and get a list of facilities that accept each type and their opening hours, so that I can find an optimal route and schedule.',
 'g04_7': 'As a user, I want to add donation centers as favorites on my 

In [9]:
# Check for duplicates
len(set(list(user_stories.values())))

51

In [10]:
sample_subset = dict(list(user_stories.items())[:5])

sample_subset

{'g04_1': 'As a user, I want to click on the address, so that it takes me to a new tab with Google Maps.',
 'g04_2': 'As a user, I want to be able to anonymously view public information, so that I know about recycling centers near me before creating an account.',
 'g04_3': 'As a user, I want to be able to enter my zip code and get a list of nearby recycling facilities, so that I can determine which ones I should consider.',
 'g04_4': 'As a user, I want to be able to get the hours of each recycling facility, so that I can arrange drop-offs on my off days or during after-work hours.',
 'g04_5': 'As a user, I want to have a flexible pick up time, so that I can more conveniently use the website.'}

In [11]:
# char = "@"
# strings_with_char = [s for s in user_stories if char in s]
# print(strings_with_char)

In [12]:
# Set up request parameters
def build_openrouter_request_data(user_story, model, or_token):
    url = "https://openrouter.ai/api/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {or_token}",
        "Content-Type": "application/json"
    }

    data = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": "You are a QA Engineer. Please generate a complete Gherkin feature file with 3-5 realistic, testable scenarios for the user story below. Please return the Gherkin only, without comments or explanations."
            },
            {
                "role": "user",
                "content": f"User Story: {user_story}"
            }
        ],
        "temperature": 0.8, 
        "provider": {
            "data_collection": "deny"
        }
    }

    return url, headers, data


In [13]:
# Synchronous request function
def openrouter_request(us_id, user_story, model, or_token):
    url, headers, data = build_openrouter_request_data(user_story, model, or_token)

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()

        created = response.json().get("created", "")
        response_content = response.json().get("choices")[0]["message"]["content"]
        prompt_tokens = response.json().get("usage", {}).get("prompt_tokens", 0)
        completion_tokens = response.json().get("usage", {}).get("completion_tokens", 0)

        return {
            "model": model,
            "created": created,
            "us_id": us_id,
            "user_story": user_story,
            "raw_response": response_content,
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
        }
    
        # return response.json()
        
    except Exception as e:
        print(f"Sync error: {e}")
        return None


In [14]:
# Asynchronous request function
async def openrouter_request_async(us_id, user_story, model, or_token):
    url, headers, data = build_openrouter_request_data(user_story, model, or_token)

    try:
        async with httpx.AsyncClient() as client:
            response = await client.post(url, headers=headers, json=data)
            response.raise_for_status()

            json_data = response.json()
            created = json_data.get("created", "")
            response_content = json_data.get("choices")[0]["message"]["content"]
            prompt_tokens = json_data.get("usage", {}).get("prompt_tokens", 0)
            completion_tokens = json_data.get("usage", {}).get("completion_tokens", 0)

            return {
                "model": model,
                "created": created,
                "us_id": us_id,
                "user_story": user_story,
                "raw_response": response_content,
                "prompt_tokens": prompt_tokens,
                "completion_tokens": completion_tokens
            }

            # return response.json()
        
    except Exception as e:
        print(f"Async error: {e}")
        return None


In [15]:
semaphore = asyncio.Semaphore(5)  # Limit concurrent requests to 5

async def limited_openrouter_request(us_id, user_story, model, or_token):
    async with semaphore:  # acquire a “slot”
        return await openrouter_request_async(us_id, user_story, model, or_token)


In [16]:
# models = ["openai/gpt-4o-mini", "meta-llama/llama-3.1-70b-instruct"]
models = ["openai/gpt-4o-mini", "google/gemini-2.0-flash-001"]

async def main():
    tasks = [
        limited_openrouter_request(us_id, user_story, model, or_token)
        for us_id, user_story in sample_subset.items()
        for model in models
    ]

    results = await asyncio.gather(*tasks)

    return results

results = await main()


In [17]:
results

[{'model': 'openai/gpt-4o-mini',
  'created': 1760351759,
  'us_id': 'g04_1',
  'user_story': 'As a user, I want to click on the address, so that it takes me to a new tab with Google Maps.',
  'raw_response': '```gherkin\nFeature: Open address in Google Maps\n\n  Scenario: User clicks on address and is redirected to Google Maps\n    Given I am on the page with an address displayed\n    When I click on the address\n    Then a new tab should open with Google Maps displaying the address\n\n  Scenario: User clicks on address with valid format\n    Given I am on the page with a valid address "1600 Amphitheatre Parkway, Mountain View, CA"\n    When I click on the address\n    Then a new tab should open with Google Maps showing "1600 Amphitheatre Parkway, Mountain View, CA"\n\n  Scenario: User clicks on address with special characters\n    Given I am on the page with an address "123 Main St, Apt #5, New York, NY"\n    When I click on the address\n    Then a new tab should open with Google Map

In [20]:
df = pd.DataFrame.from_records(results)

In [22]:
df.head()

,model,created,us_id,user_story,raw_response,prompt_tokens,completion_tokens
0,openai/gpt-4o-mini,1760351759,g04_1,"As a user, I want to click on the address, so ...",```gherkin\nFeature: Open address in Google Ma...,83,312
1,google/gemini-2.0-flash-001,1760351759,g04_1,"As a user, I want to click on the address, so ...",```gherkin\nFeature: Address Link Opens Google...,70,271
2,openai/gpt-4o-mini,1760351759,g04_2,"As a user, I want to be able to anonymously vi...",```gherkin\nFeature: Anonymous Access to Publi...,87,361
3,google/gemini-2.0-flash-001,1760351759,g04_2,"As a user, I want to be able to anonymously vi...",```gherkin\nFeature: Anonymous Access to Recyc...,74,357
4,openai/gpt-4o-mini,1760351759,g04_3,"As a user, I want to be able to enter my zip c...",```gherkin\nFeature: Nearby Recycling Faciliti...,92,370


In [ ]:
df.to_csv(f"gherkin_generation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv", index=False)

## For multi-turn chats

In [ ]:
LOG_DIR = "./multiturn_logs/exp2_attempt4"

In [ ]:
# Save multi-turn chat to timestamped JSON file
def save_conversation(conversation_log, model):
    filename = f"{LOG_DIR}/{model.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(conversation_log, f, indent=2, ensure_ascii=False)
        
    print(f"Saved conversation for {model} to {filename}")

    return filename

In [ ]:
# Find path to the latest chat log for a specific model
def find_latest_conversation(model):
    model = model.replace('/', '_')

    files = [
        f for f in os.listdir(LOG_DIR)
        if f.startswith(model) and f.endswith(".json")
    ]

    if not files:
        return None
    
    files.sort(reverse=True)
    return os.path.join(LOG_DIR, files[0])

In [ ]:
# prompt = "You are a QA Engineer. For each user story I give you, please generate a complete Gherkin feature file with at least three realistic, testable scenarios. Try to cover: 1. The happy path (expected successful flow), 2. At least one edge case, 3. At least one error or failure condition. Please return the Gherkin only, without comments or explanation."
# reminder = "Reminder: your task is to generate a complete Gherkin feature file with at least three realistic, testable scenarios for the user story I give you, returning only the Gherkin."

In [ ]:
prompt = "You are a QA Engineer. For each user story I give you, please generate a complete Gherkin feature file with 3-5 realistic, testable scenarios. Please return the Gherkin only, without comments or explanation."
reminder = None

In [ ]:
#  Load latest chat log from JSON and rebuild messages
def load_conversation(filename):
    with open(filename, "r", encoding="utf-8") as f:
        conversation_log = json.load(f)

    messages = [
        {"role": "system", "content": (
            prompt
        )}
    ]

    for turn in conversation_log["conversation"]:
        messages.append({"role": "user", "content": turn["user_story"]})
        messages.append({"role": "assistant", "content": turn["assistant_response"]})

    completed_stories = [turn["us_id"] for turn in conversation_log["conversation"]]
    
    return messages, conversation_log, completed_stories

In [ ]:
print(datetime.now().isoformat())

In [ ]:
async def chat_with_model(model, user_stories, or_token):
    """Run a multi-turn chat for one model, automatically resuming if a saved log exists."""
    headers = {
        "Authorization": f"Bearer {or_token}",
        "Content-Type": "application/json"
    }

    log_file = find_latest_conversation(model)

    if log_file:
        print(f"Resuming from {log_file}")

        messages, conversation_log, completed_stories = load_conversation(log_file)
    else:
        print(f"Starting new chat for {model}")

        messages = [
            {
                "role": "system", "content": (
                    prompt
            )}
        ]
        
        conversation_log = {
            "model": model,
            "timestamp": datetime.now().isoformat(),
            "conversation": []
        }

        completed_stories = []

    async with httpx.AsyncClient() as client:
        for us_id, story in user_stories.items():
            if us_id in completed_stories:
                print(f"Skipping completed story: {us_id}")

                continue

            if reminder:
                messages.append({"role": "user", "content": reminder})

            messages.append({"role": "user", "content": story})

            response = await client.post(
                url="https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json={
                    "model": model,
                    "messages": messages,
                    "temperature": 0.8,
                    "provider": {
                        "data_collection": "deny"
                        }
                }
            )

            data = response.json()
            reply = data["choices"][0]["message"]["content"]
            messages.append({"role": "assistant", "content": reply})

            conversation_log["conversation"].append({
                "us_id": us_id,
                "user_story": story,
                "assistant_response": reply,
                "raw_response": data
            })

            save_conversation(conversation_log, model)
            await asyncio.sleep(1)  # short pause for rate limits

    return conversation_log

In [ ]:
async def main():
    results = await asyncio.gather(*[
        chat_with_model(model, sample_subset, or_token)
        for model in models
    ])
    return results


results = await main()

In [ ]:
len(results) # Count of models processed

In [19]:
results

[{'model': 'openai/gpt-4o-mini',
  'created': 1760351759,
  'us_id': 'g04_1',
  'user_story': 'As a user, I want to click on the address, so that it takes me to a new tab with Google Maps.',
  'raw_response': '```gherkin\nFeature: Open address in Google Maps\n\n  Scenario: User clicks on address and is redirected to Google Maps\n    Given I am on the page with an address displayed\n    When I click on the address\n    Then a new tab should open with Google Maps displaying the address\n\n  Scenario: User clicks on address with valid format\n    Given I am on the page with a valid address "1600 Amphitheatre Parkway, Mountain View, CA"\n    When I click on the address\n    Then a new tab should open with Google Maps showing "1600 Amphitheatre Parkway, Mountain View, CA"\n\n  Scenario: User clicks on address with special characters\n    Given I am on the page with an address "123 Main St, Apt #5, New York, NY"\n    When I click on the address\n    Then a new tab should open with Google Map

In [18]:
rows = []

for model_result in results:
    model = model_result.get("model")
    timestamp = model_result.get("timestamp")
    conversations = model_result.get("conversation", [])
    
    for conv in conversations:
        us_id = conv.get("us_id")
        user_story = conv.get("user_story")
        assistant_response = conv.get("assistant_response")

        created = conv.get("raw_response", {}).get("created")
        # Token counts (if available)
        usage = conv.get("raw_response", {}).get("usage", {})
        prompt_tokens = usage.get("prompt_tokens")
        completion_tokens = usage.get("completion_tokens")
        # total_tokens = usage.get("total_tokens")

        rows.append({
            "model": model,
            "timestamp": timestamp,
            "us_id": us_id,
            "user_story": user_story,
            "assistant_response": assistant_response,
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
            "created": created
            # "total_tokens": total_tokens
        })

# Create DataFrame
df = pd.DataFrame(rows)

# # Optional: order columns
# df = df[
#     [
#         "model",
#         "timestamp",
#         "us_id",
#         "user_story",
#         "assistant_response",
#         "prompt_tokens",
#         "completion_tokens",
#         "total_tokens"
#     ]
# ]

# Preview
df.head()


""


In [ ]:
df.iloc[0]["assistant_response"]

In [ ]:
df.iloc[5]["assistant_response"]

In [ ]:
df.head(10)

In [ ]:
df.to_csv('exp2_p4_g04_10-10-25_first_5_us.csv', index=False, mode='w', header=True)

In [ ]:
# Remove None results i.e. requests that failed
filtered_results = [r for r in results if r is not None]

In [ ]:
len(filtered_results)

In [ ]:
df = pd.DataFrame(filtered_results)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Identify user stories that only appear once in the dataset i.e. those for which a model request failed
story_counts = df["user_story"].value_counts()
missing_stories = story_counts[story_counts == 1].index.tolist()

missing_stories_df = df[df["user_story"].isin(missing_stories)]
missing_stories_df.head(10)


In [ ]:
# Request missing Gherkin for user stories that only appear once in the dataset
results = []

for index, row in missing_stories_df.iterrows():
    user_story = row['user_story']
    model = row['model']
    missing_model = (set(models) - {model}).pop()  # Get the other model

    print(f"Requesting missing Gherkin for user story: {row['user_story']} using model: {missing_model}")

    result = openrouter_request(user_story, missing_model, or_token)
    results.append(result)


In [ ]:
results

In [ ]:
df.shape

In [ ]:
df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)

In [ ]:
df.shape

In [ ]:
df["app"] = "g04-recycling"

In [ ]:
df.head()

In [ ]:
df["timestamp"].value_counts()

In [ ]:
df.drop(columns=["timestamp"], inplace=True)

In [ ]:
df.to_csv('s1_p1_g04_9-10-25.csv', index=False, mode='w', header=True)

In [ ]:
df.shape

In [ ]:
# Add file names to the DataFrame
# df["file"] = df["user_story"].map(user_stories_dict)

In [ ]:
df.head()

In [ ]:
df.isna().sum()